In [1]:
# EOMs
%reset
#import libraries
import sympy as sym
import numpy as np
import pickle as pickle

sym.init_printing()
from IPython.display import display #for pretty printing

# Symbols --------------------------------------------------
links = ['body','UL','UR','footL','footR']

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [2]:
#Parameters
masses = {}
lengths = {}
inertias = {}
th = {}
dth = {}
ddth = {}
r_prismatic = {}
dr_prismatic = {}
ddr_prismatic = {}

g = sym.symbols('g')

#gen coordinates
x_body,y_body,th_body = sym.symbols(['x_body','y_body','theta_{0}'.format('body')])
dx_body,dy_body,dth_body = sym.symbols(['\dot{\\x}_{body}','\dot{\\y}_{body}','\dot{\\theta}_{body}'])
ddx_body,ddy_body,ddth_body = sym.symbols(['\ddot{\\x}_{body}','\ddot{\\y}_{body}','\ddot{\\theta}_{body}'])

for link in links:
    masses.update({(link): sym.symbols('m_{0}'.format(link))})
    lengths.update({(link): sym.symbols('l_{0}'.format(link))})
    inertias.update({(link): sym.symbols('In_{0}'.format(link))})
    
    if 'body' in link:
        pass
    if 'foot' in link:
        r_prismatic.update({(link): sym.symbols('{{r\_prismatic}}_{{{0}}}'.format(link))})
        dr_prismatic.update({(link): sym.symbols('\dot{{\\r\_prismatic}}_{{{0}}}'.format(link))})
        ddr_prismatic.update({(link): sym.symbols('\ddot{{\\r\_prismatic}}_{{{0}}}'.format(link))})
    else:
        th.update({(link): sym.symbols('theta_{0}'.format(link))}) 
        dth.update({(link): sym.symbols('\dot{{\\theta}}_{{{0}}}'.format(link))})
        ddth.update({(link): sym.symbols('\ddot{{\\theta}}_{{{0}}}'.format(link))})

In [3]:
q = sym.Matrix([x_body]+[y_body]+[th[link] for link in links if 'foot' not in link]+[r_prismatic[link] for link in links if 'foot' in link])
dq = sym.Matrix([dx_body]+[dy_body]+[dth[link] for link in links if 'foot' not in link]+[dr_prismatic[link] for link in links if 'foot' in link])
ddq = sym.Matrix([ddx_body]+[ddy_body]+[ddth[link] for link in links if 'foot' not in link]+[ddr_prismatic[link] for link in links if 'foot' in link])

In [4]:
q

⎡        x_body        ⎤
⎢                      ⎥
⎢        y_body        ⎥
⎢                      ⎥
⎢        θ_body        ⎥
⎢                      ⎥
⎢         θ_UL         ⎥
⎢                      ⎥
⎢         θ_UR         ⎥
⎢                      ⎥
⎢{r\_prismatic}_{footL}⎥
⎢                      ⎥
⎣{r\_prismatic}_{footR}⎦

In [5]:
dq

⎡      \dot{\x}_{body}      ⎤
⎢                           ⎥
⎢      \dot{\y}_{body}      ⎥
⎢                           ⎥
⎢    \dot{\theta}_{body}    ⎥
⎢                           ⎥
⎢     \dot{\theta}_{UL}     ⎥
⎢                           ⎥
⎢     \dot{\theta}_{UR}     ⎥
⎢                           ⎥
⎢\dot{\r\_prismatic}_{footL}⎥
⎢                           ⎥
⎣\dot{\r\_prismatic}_{footR}⎦

In [6]:
ddq

⎡      \ddot{\x}_{body}      ⎤
⎢                            ⎥
⎢      \ddot{\y}_{body}      ⎥
⎢                            ⎥
⎢    \ddot{\theta}_{body}    ⎥
⎢                            ⎥
⎢     \ddot{\theta}_{UL}     ⎥
⎢                            ⎥
⎢     \ddot{\theta}_{UR}     ⎥
⎢                            ⎥
⎢\ddot{\r\_prismatic}_{footL}⎥
⎢                            ⎥
⎣\ddot{\r\_prismatic}_{footR}⎦

In [7]:
''' Setup positions (to COM) and velocities'''


# define absolute angles
abs_angles = {}
r_links = {}
r_force = {}
   
abs_angles.update({('ULA'): th_body + th['UL']})
abs_angles.update({('URA'): th_body + th['UR']})

r_links.update({'body': sym.Matrix([[x_body,
                                     y_body,
                                     th['body']]]).transpose()})

r_links.update({('UL'): sym.Matrix([[x_body-lengths['UL']/2*sym.cos(abs_angles['ULA']),
                                     y_body-lengths['UL']/2*sym.sin(abs_angles['ULA']),
                                     abs_angles['ULA']]]).transpose()})

r_links.update({('UR'): sym.Matrix([[x_body+lengths['UR']/2*sym.cos(np.pi - abs_angles['URA']),
                                     y_body-lengths['UR']/2*sym.sin(np.pi - abs_angles['URA']),
                                     abs_angles['URA']]]).transpose()})

r_links.update({('footL'): sym.Matrix([[x_body-(lengths['UL']+r_prismatic['footL'])*sym.cos(abs_angles['ULA']),
                                        y_body-(lengths['UL']+r_prismatic['footL'])*sym.sin(abs_angles['ULA']),
                                        0.0]]).transpose()})

r_links.update({('footR'): sym.Matrix([[x_body+(lengths['UR']+r_prismatic['footR'])*sym.cos(np.pi - abs_angles['URA']),
                                        y_body-(lengths['UR']+r_prismatic['footR'])*sym.sin(np.pi - abs_angles['URA']),
                                        0.0]]).transpose()})

r_force.update({('left'): sym.Matrix([[x_body-lengths['UL']*sym.cos(abs_angles['ULA']),
                                       y_body-lengths['UL']*sym.sin(abs_angles['ULA']),
                                       0.0]]).transpose()})

r_force.update({('right'): sym.Matrix([[x_body+lengths['UR']*sym.cos(np.pi-abs_angles['URA']),
                                        y_body-lengths['UR']*sym.sin(np.pi-abs_angles['URA']),
                                        0.0]]).transpose()})

In [8]:
# the Jacobians
Jbns = {}
dr_links = {}

for link in links:
    Jbns.update({(link): r_links[link].jacobian(q)})
    dr_links.update({(link): Jbns[link]*dq})
    
Jbn_left_force = r_force['left'].jacobian(q)
Jbn_right_force = r_force['right'].jacobian(q)
    
Mass_matrix = {}
Ttot = sym.Matrix([[0.0]])
Vtot = 0.0

for link in links:
    Mass_matrix.update({(link): sym.Matrix([[masses[link],masses[link],inertias[link]]])})
    Ttot = Ttot + 0.5*Mass_matrix[link]*sym.matrix_multiply_elementwise(dr_links[link],dr_links[link])
    Vtot = Vtot + masses[link]*g*r_links[link][1]
Ttot = Ttot[0]

In [9]:
# Manipulator Equation (M*ddq + C + G = Tau + Q)

# M Matrix
M = sym.zeros(len(q),len(q))

for i in range(len(q)):
    for j in range(len(q)):
        M[i,j] = sym.diff(sym.diff(Ttot,dq[i]),dq[j])

M = sym.simplify(M)

M_size = int(np.sqrt(len(M))) # assumes square matrix (len(M) = 4x4 = 16)

dM = sym.zeros(M_size,M_size)

for i in range(M_size):
    for j in range(M_size):
        dM[i,j] = sym.Matrix([M[i,j]]).jacobian(q)*dq

# C Matrix
C = dM*dq - (sym.Matrix([Ttot]).jacobian(q)).transpose() # 4x1 - 1x4^Transpose

# G Matrix
G = (sym.Matrix([Vtot]).jacobian(q)).transpose() # 4x1

# define 
tau = {}
grf = {}
fp = {}

tau.update({('left'): sym.symbols('tau_left')})
tau.update({('right'): sym.symbols('tau_right')})

fp.update({('left'): sym.symbols('Fp_left')})
fp.update({('right'): sym.symbols('Fp_right')})

grf.update({('x','left'): sym.symbols('GRF_x_left')})
grf.update({('y','left'): sym.symbols('GRF_y_left')})
grf.update({('x','right'): sym.symbols('GRF_x_right')})
grf.update({('y','right'): sym.symbols('GRF_y_right')})

# Q?
# GRF_left = sym.Matrix([GRF['x','left'],GRF['y','left']])
# GRF_right = sym.Matrix([GRF['x','right'],GRF['y','right']])
        
left_momentum = -(0.5*lengths['UL']+r_prismatic['footL'])*sym.cos(abs_angles['ULA'])*grf['y','left'] +\
                 (0.5*lengths['UL']+r_prismatic['footL'])*sym.sin(abs_angles['ULA'])*grf['x','left'] 
GRF_left = sym.Matrix([[grf['x','left']],[grf['y','left']],[left_momentum*0.0]])

right_momentum = (0.5*lengths['UR']+r_prismatic['footR'])*sym.cos(np.pi-abs_angles['URA'])*grf['y','right'] +\
                 (0.5*lengths['UR']+r_prismatic['footR'])*sym.sin(np.pi-abs_angles['URA'])*grf['x','right'] 
GRF_right = sym.Matrix([[grf['x','right']],[grf['y','right']],[right_momentum*0.0]])

Tau = sym.Matrix([[0],[0],[0],[tau['left']],[tau['right']],[0],[0]])
# Fp = sym.Matrix([[0],[0],[0],[0],[0],[fp['left']],[fp['right']]])

Fp_UL = sym.Matrix([[-fp['left']*sym.cos(abs_angles['ULA'])],[-fp['left']*sym.sin(abs_angles['ULA'])],[0.0]])
Fp_LL = sym.Matrix([[fp['left']*sym.cos(abs_angles['ULA'])],[fp['left']*sym.sin(abs_angles['ULA'])],[0.0]])

Fp_UR = sym.Matrix([[fp['right']*sym.cos(np.pi-abs_angles['URA'])],[-fp['right']*sym.sin(np.pi-abs_angles['URA'])],[0.0]])
Fp_LR = sym.Matrix([[-fp['right']*sym.cos(np.pi-abs_angles['URA'])],[fp['right']*sym.sin(np.pi-abs_angles['URA'])],[0.0]])

# transpose both terms
Q = Jbns['footL'].transpose()*GRF_left + \
    Jbns['footR'].transpose()*GRF_right + \
    Jbns['footL'].transpose()*Fp_LL + \
    Jbns['footR'].transpose()*Fp_LR + \
    Jbn_left_force.transpose()*Fp_UL +\
    Jbn_right_force.transpose()*Fp_UR

In [10]:
# Manipulator Equation 
EOM = M*ddq + G + C - Tau - Q 

# save the EOM in a file (saving the EOM saves lots of time)
import dill
dill.dump_session('M1_unsimplified_zero.db')

In [11]:
EOMs = sym.zeros(len(q),1)
for j in range(len(q)):
    EOMs[j] = EOM[j].simplify()

In [12]:
# save the EOM in a file (saving the EOM saves lots of time)
import dill
dill.dump_session('M1_simplified_zero.db')